In [1]:
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping

import json
import numpy as np
import pandas as pd
import tensorflow as tf
import os

In [2]:
with open('json/string_test.json', encoding='UTF8') as file:
    dataset = json.load(file)

In [3]:
print(dataset[1])

{'지역': '대구', '계열': '자연', '대학명': 'DGIST', '학과명': '융복합대학(기초학부)', '전형유형': '종합', '전형명': '학교장추천', '최저적용유무': '', '최초지원결과': '합', '최종지원결과': '합', '전과목(평균)': '1.47', '국영수사(평균)': '1.51', '국영수과(평균)': '1.29', '국영수탐(백분위)': '281.5', '국영수탐(등급)': '1.67'}


In [4]:
# JSON 데이터를 pandas DataFrame으로 변환
df = pd.DataFrame(dataset)

# "최저 지원 유무" 열 삭제
df_modify = df.drop(columns=['최저적용유무', '최초지원결과'])

# 빈 값이 있는 행 제거
df_modify = df_modify.replace('', pd.NA).dropna()

# 결과를 다시 JSON 형식으로 변환
modified_data = df_modify.to_dict(orient='records')

In [5]:
print(modified_data[1])

{'지역': '대구', '계열': '자연', '대학명': 'DGIST', '학과명': '융복합대학(기초학부)', '전형유형': '종합', '전형명': '학교장추천', '최종지원결과': '합', '전과목(평균)': '1.47', '국영수사(평균)': '1.51', '국영수과(평균)': '1.29', '국영수탐(백분위)': '281.5', '국영수탐(등급)': '1.67'}


In [6]:
# 숫자로 변환하여 NumPy 배열 생성
numeric_data = np.array([
    [data['전과목(평균)'],
     data['국영수사(평균)'],
     data['국영수과(평균)'],
     data['국영수탐(백분위)'],
     data['국영수탐(등급)']] for data in modified_data])

# 문자열 데이터를 numpy 배열로 변환 (hashing trick 적용) --> 수정 필요
string_data = np.array([
    [data['지역'], 
     data['계열'], 
     data['대학명'], 
     data['학과명'], 
     data['전형유형'], 
     data['전형명']] for data in modified_data])

# 결과값을 정수로 매핑 ("최종지원결과"를 1 또는 0으로 변환)
result = np.array([data['최종지원결과'] for data in modified_data])

In [7]:
print(numeric_data[0:5])
print(result[0:5])

[['1.32' '1.31' '1.22' '252.5' '2.67']
 ['1.47' '1.51' '1.29' '281.5' '1.67']
 ['1.3' '1.3' '1.3' '225.5' '3.67']
 ['1.39' '1.44' '1.33' '224' '3.33']
 ['1.59' '1.41' '1.36' '263.5' '2.17']]
['합' '합' '합' '합' '불']


In [8]:
# 2차원 배열의 각 행에 대해 문자열을 정수로 변환
X = np.array(numeric_data).astype(float)

# y값 문자열에 대해 원-핫 인코딩
y = np.array(result)
e = LabelEncoder()
e.fit(y)
Y = e.transform(y)

print(X[0:5])
print(Y[0:5])


[[  1.32   1.31   1.22 252.5    2.67]
 [  1.47   1.51   1.29 281.5    1.67]
 [  1.3    1.3    1.3  225.5    3.67]
 [  1.39   1.44   1.33 224.     3.33]
 [  1.59   1.41   1.36 263.5    2.17]]
[1 1 1 1 0]


In [9]:
model = tf.keras.models.Sequential([
    # 숫자 같은 경우 계속 돌려 보면서 정확도가 높은 방향으로 모델을 튜닝시켜 주는 작업이 필요함
    tf.keras.layers.Dense(64, activation='tanh'),
    tf.keras.layers.Dense(128, activation='tanh'),
    # 마지막 레이어는 예측결과(sigmoid -> 0~1확률)
    tf.keras.layers.Dense(1, activation='sigmoid'),
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 학습 데이터, 실제 정답
model.fit(np.array(X), np.array(Y), epochs=100)



Epoch 1/100


2/2 [==============================] - 1s 5ms/step - loss: 0.8105 - accuracy: 0.4667
Epoch 2/100
2/2 [==============================] - 0s 3ms/step - loss: 0.4805 - accuracy: 0.8333
Epoch 3/100
2/2 [==============================] - 0s 3ms/step - loss: 0.4748 - accuracy: 0.8333
Epoch 4/100
2/2 [==============================] - 0s 3ms/step - loss: 0.5133 - accuracy: 0.8333
Epoch 5/100
2/2 [==============================] - 0s 2ms/step - loss: 0.5352 - accuracy: 0.8333
Epoch 6/100
2/2 [==============================] - 0s 2ms/step - loss: 0.5281 - accuracy: 0.8333
Epoch 7/100
2/2 [==============================] - 0s 3ms/step - loss: 0.5093 - accuracy: 0.8333
Epoch 8/100
2/2 [==============================] - 0s 3ms/step - loss: 0.4779 - accuracy: 0.8333
Epoch 9/100
2/2 [==============================] - 0s 3ms/step - loss: 0.4537 - accuracy: 0.8333
Epoch 10/100
2/2 [==============================] - 0s 2ms/step - loss: 0.4545 - accuracy: 0.8333
Epoch 11/100
2/2 [=======

In [10]:
# 모델 생성
model = Sequential()
model.add(Dense(64, activation='tanh'))
model.add(Dense(128, activation='tanh')) 
# 마지막 레이어는 예측결과(sigmoid -> 0~1확률)
model.add(Dense(1, activation='sigmoid')) 

# 모델 컴파일
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# 모델 실행
model.fit(np.array(X), np.array(Y), epochs=100)

Epoch 1/100
2/2 [==============================] - 0s 2ms/step - loss: 0.6490 - accuracy: 0.5000
Epoch 2/100
2/2 [==============================] - 0s 2ms/step - loss: 0.4616 - accuracy: 0.8333
Epoch 3/100
2/2 [==============================] - 0s 2ms/step - loss: 0.4771 - accuracy: 0.8333
Epoch 4/100
2/2 [==============================] - 0s 2ms/step - loss: 0.5211 - accuracy: 0.8333
Epoch 5/100
2/2 [==============================] - 0s 2ms/step - loss: 0.5188 - accuracy: 0.8333
Epoch 6/100
2/2 [==============================] - 0s 2ms/step - loss: 0.4960 - accuracy: 0.8333
Epoch 7/100
2/2 [==============================] - 0s 3ms/step - loss: 0.4596 - accuracy: 0.8333
Epoch 8/100
2/2 [==============================] - 0s 2ms/step - loss: 0.4503 - accuracy: 0.8333
Epoch 9/100
2/2 [==============================] - 0s 2ms/step - loss: 0.4569 - accuracy: 0.8333
Epoch 10/100
2/2 [==============================] - 0s 2ms/step - loss: 0.4748 - accuracy: 0.8333
Epoch 11/100
2/2 [===========

In [11]:
# 예측
predict = model.predict([ [1.32, 1.31, 1.22, 252.5, 2.67], [5, 4.5, 4.8, 72, 6.2] ])
print(predict)

1/1 [==============================] - 0s 72ms/step
[[0.6163709 ]
 [0.00362388]]


In [20]:
# 모델 생성
model = Sequential()
model.add(Dense(64, activation='tanh'))
model.add(Dense(128, activation='tanh')) 
# 마지막 레이어는 예측결과(sigmoid -> 0~1확률)
model.add(Dense(1, activation='sigmoid')) 

# 모델 컴파일
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# 모델 저장 폴더 만들기
MODEL_DIR = './model/'
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)
    
modelpath = "./model/best_model.h5"

# 모델 업데이트 및 저장(앞에 모델보다 나아졌을때만 저장)
checkpointer = ModelCheckpoint(filepath=modelpath, moniter='loss', verbos=1, save_best_only=True)

# 학습 자동 중단 설정(학습을 계속해도 정확도에 진전이 없다면 stop)
early_stopping_callback = EarlyStopping(monitor='loss', patience=100)

# 모델 실행
model.fit(np.array(X), np.array(Y), epochs=100, batch_size=32, validation_data=(np.array(X), np.array(Y)), verbose=0, callbacks=[early_stopping_callback, checkpointer])